# 🔍 Notebook 11: Text Embeddings a Sémantické Vyhledávání

**Autor:** Praut s.r.o. - AI Integration & Business Automation

V tomto notebooku se naučíme pracovat s textovými embeddings - vektorovými reprezentacemi textu. Ukážeme si jak vytvářet embeddings, provádět sémantické vyhledávání a budovat jednoduché RAG systémy.

## Co se naučíte:
- Vytváření text embeddings pomocí Sentence Transformers
- Sémantické vyhledávání a similarity search
- Clustering dokumentů
- Jednoduché vektorové databáze
- Základy RAG (Retrieval Augmented Generation)

In [ ]:
# Instalace potřebných knihoven
!pip install -q sentence-transformers faiss-cpu numpy pandas scikit-learn matplotlib

In [ ]:
import torch
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import warnings
warnings.filterwarnings('ignore')

# Detekce zařízení
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Používám zařízení: {device}")
if device == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 1. Základy Text Embeddings

In [ ]:
# Načtení modelu pro embeddings
# all-MiniLM-L6-v2 je rychlý a kvalitní model pro obecné použití
model = SentenceTransformer('all-MiniLM-L6-v2', device=device)

print(f"✅ Model načten")
print(f"   Dimenze embeddings: {model.get_sentence_embedding_dimension()}")
print(f"   Max délka sekvence: {model.max_seq_length}")

In [ ]:
# Vytvoření embeddings pro jednotlivé věty
sentences = [
    "Python je populární programovací jazyk.",
    "JavaScript se používá pro webový vývoj.",
    "Kočky jsou oblíbená domácí zvířata.",
    "Programování vyžaduje logické myšlení.",
    "Psi jsou věrní společníci člověka."
]

# Vytvoření embeddings
embeddings = model.encode(sentences)

print(f"Počet vět: {len(sentences)}")
print(f"Tvar embeddings: {embeddings.shape}")
print(f"\nUkázka prvního embeddingu (prvních 10 hodnot):")
print(embeddings[0][:10])

In [ ]:
# Výpočet podobnosti mezi větami
similarity_matrix = cosine_similarity(embeddings)

# Vizualizace
plt.figure(figsize=(10, 8))
plt.imshow(similarity_matrix, cmap='RdYlGn', aspect='auto')
plt.colorbar(label='Cosine Similarity')
plt.xticks(range(len(sentences)), [s[:30] + '...' for s in sentences], rotation=45, ha='right')
plt.yticks(range(len(sentences)), [s[:30] + '...' for s in sentences])
plt.title('Matice podobnosti vět')

# Přidání hodnot do buněk
for i in range(len(sentences)):
    for j in range(len(sentences)):
        plt.text(j, i, f'{similarity_matrix[i, j]:.2f}', 
                ha='center', va='center', fontsize=9)

plt.tight_layout()
plt.show()

## 2. Sémantické vyhledávání

In [ ]:
class SemanticSearch:
    """
    Třída pro sémantické vyhledávání v korpusu dokumentů.
    """
    
    def __init__(self, model_name='all-MiniLM-L6-v2'):
        self.model = SentenceTransformer(model_name, device=device)
        self.documents = []
        self.embeddings = None
        self.metadata = []
    
    def add_documents(self, documents, metadata=None):
        """
        Přidá dokumenty do indexu.
        
        Args:
            documents: Seznam textových dokumentů
            metadata: Volitelná metadata pro každý dokument
        """
        self.documents.extend(documents)
        
        if metadata:
            self.metadata.extend(metadata)
        else:
            self.metadata.extend([{} for _ in documents])
        
        # Vytvoření embeddings
        new_embeddings = self.model.encode(documents, show_progress_bar=True)
        
        if self.embeddings is None:
            self.embeddings = new_embeddings
        else:
            self.embeddings = np.vstack([self.embeddings, new_embeddings])
        
        print(f"Přidáno {len(documents)} dokumentů. Celkem: {len(self.documents)}")
    
    def search(self, query, top_k=5):
        """
        Vyhledá nejpodobnější dokumenty k dotazu.
        """
        if self.embeddings is None or len(self.documents) == 0:
            return []
        
        # Embedding dotazu
        query_embedding = self.model.encode([query])
        
        # Výpočet podobnosti
        similarities = cosine_similarity(query_embedding, self.embeddings)[0]
        
        # Seřazení podle podobnosti
        top_indices = similarities.argsort()[-top_k:][::-1]
        
        results = []
        for idx in top_indices:
            results.append({
                "document": self.documents[idx],
                "score": float(similarities[idx]),
                "metadata": self.metadata[idx],
                "index": int(idx)
            })
        
        return results
    
    def find_similar(self, document_index, top_k=5):
        """
        Najde dokumenty podobné danému dokumentu.
        """
        if document_index >= len(self.documents):
            return []
        
        doc_embedding = self.embeddings[document_index:document_index+1]
        similarities = cosine_similarity(doc_embedding, self.embeddings)[0]
        
        # Vynechat samotný dokument
        similarities[document_index] = -1
        
        top_indices = similarities.argsort()[-top_k:][::-1]
        
        return [
            {
                "document": self.documents[idx],
                "score": float(similarities[idx]),
                "index": int(idx)
            }
            for idx in top_indices
        ]

# Test
search_engine = SemanticSearch()
print("✅ SemanticSearch inicializován")

In [ ]:
# Příprava testovacího korpusu
documents = [
    "Python je interpretovaný programovací jazyk vysoké úrovně s dynamickým typováním.",
    "Machine learning je podoblast umělé inteligence zabývající se algoritmy, které se učí z dat.",
    "Django je webový framework pro Python, který umožňuje rychlý vývoj webových aplikací.",
    "TensorFlow je open-source knihovna pro numerické výpočty a strojové učení.",
    "JavaScript je skriptovací jazyk běžně používaný pro interaktivní webové stránky.",
    "React je JavaScript knihovna pro tvorbu uživatelských rozhraní.",
    "PostgreSQL je výkonná open-source relační databáze.",
    "Docker je platforma pro kontejnerizaci aplikací.",
    "Kubernetes orchestruje kontejnerizované aplikace v clusteru.",
    "Git je distribuovaný systém správy verzí pro sledování změn v souborech.",
    "REST API je architektonický styl pro návrh webových služeb.",
    "GraphQL je dotazovací jazyk pro API vyvinutý Facebookem.",
    "Redis je in-memory databáze často používaná jako cache.",
    "Celery je distribuovaná fronta úloh pro Python.",
    "Angular je TypeScript framework pro tvorbu webových aplikací."
]

metadata = [
    {"category": "programming", "language": "python"},
    {"category": "ai", "topic": "ml"},
    {"category": "web", "language": "python"},
    {"category": "ai", "topic": "ml"},
    {"category": "programming", "language": "javascript"},
    {"category": "web", "language": "javascript"},
    {"category": "database", "type": "relational"},
    {"category": "devops", "topic": "containers"},
    {"category": "devops", "topic": "orchestration"},
    {"category": "devops", "topic": "version-control"},
    {"category": "web", "topic": "api"},
    {"category": "web", "topic": "api"},
    {"category": "database", "type": "cache"},
    {"category": "devops", "language": "python"},
    {"category": "web", "language": "typescript"}
]

# Přidání dokumentů
search_engine.add_documents(documents, metadata)

In [ ]:
# Test vyhledávání
queries = [
    "Jak vytvořit webovou aplikaci v Pythonu?",
    "Nástroje pro správu kontejnerů",
    "Knihovny pro strojové učení"
]

for query in queries:
    print(f"\n🔍 Dotaz: {query}")
    print("-" * 50)
    
    results = search_engine.search(query, top_k=3)
    
    for i, result in enumerate(results, 1):
        print(f"{i}. [{result['score']:.3f}] {result['document'][:80]}...")
        if result['metadata']:
            print(f"   Metadata: {result['metadata']}")

## 3. Vícejazyčné embeddings

In [ ]:
# Vícejazyčný model
multilingual_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2', device=device)

print("✅ Vícejazyčný model načten")

In [ ]:
# Test cross-lingual similarity
sentences_multilingual = [
    "The cat is sitting on the mat.",          # EN
    "Kočka sedí na koberci.",                   # CZ
    "Die Katze sitzt auf der Matte.",          # DE
    "Le chat est assis sur le tapis.",         # FR
    "The dog is running in the park.",         # EN
    "Pes běží v parku.",                        # CZ
]

# Embeddings
multi_embeddings = multilingual_model.encode(sentences_multilingual)

# Matice podobnosti
multi_sim = cosine_similarity(multi_embeddings)

# Vizualizace
plt.figure(figsize=(10, 8))
plt.imshow(multi_sim, cmap='RdYlGn')
plt.colorbar(label='Similarity')

labels = ['Cat (EN)', 'Kočka (CZ)', 'Katze (DE)', 'Chat (FR)', 'Dog (EN)', 'Pes (CZ)']
plt.xticks(range(len(labels)), labels, rotation=45, ha='right')
plt.yticks(range(len(labels)), labels)
plt.title('Cross-lingual Similarity')

for i in range(len(labels)):
    for j in range(len(labels)):
        plt.text(j, i, f'{multi_sim[i, j]:.2f}', ha='center', va='center', fontsize=9)

plt.tight_layout()
plt.show()

## 4. Clustering dokumentů

In [ ]:
class DocumentClusterer:
    """
    Clustering dokumentů pomocí embeddings.
    """
    
    def __init__(self, model_name='all-MiniLM-L6-v2'):
        self.model = SentenceTransformer(model_name, device=device)
    
    def cluster(self, documents, n_clusters=3):
        """
        Seskupí dokumenty do clusterů.
        """
        # Vytvoření embeddings
        embeddings = self.model.encode(documents, show_progress_bar=True)
        
        # K-Means clustering
        kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
        cluster_labels = kmeans.fit_predict(embeddings)
        
        # Organizace výsledků
        clusters = {i: [] for i in range(n_clusters)}
        for doc, label in zip(documents, cluster_labels):
            clusters[label].append(doc)
        
        return {
            "clusters": clusters,
            "labels": cluster_labels,
            "embeddings": embeddings,
            "centroids": kmeans.cluster_centers_
        }
    
    def visualize_clusters(self, embeddings, labels, documents=None):
        """
        Vizualizuje clustery pomocí t-SNE.
        """
        # Redukce dimenzí
        tsne = TSNE(n_components=2, random_state=42, perplexity=min(30, len(embeddings)-1))
        reduced = tsne.fit_transform(embeddings)
        
        # Vizualizace
        plt.figure(figsize=(12, 8))
        scatter = plt.scatter(reduced[:, 0], reduced[:, 1], c=labels, cmap='viridis', s=100, alpha=0.7)
        plt.colorbar(scatter, label='Cluster')
        
        if documents:
            for i, doc in enumerate(documents):
                plt.annotate(doc[:20] + '...', (reduced[i, 0], reduced[i, 1]), 
                           fontsize=8, alpha=0.7)
        
        plt.title('Document Clusters (t-SNE visualization)')
        plt.xlabel('t-SNE 1')
        plt.ylabel('t-SNE 2')
        plt.tight_layout()
        plt.show()

# Test
clusterer = DocumentClusterer()
print("✅ DocumentClusterer připraven")

In [ ]:
# Test clusteringu
result = clusterer.cluster(documents, n_clusters=4)

print("Výsledky clusteringu:")
print("=" * 50)

for cluster_id, docs in result['clusters'].items():
    print(f"\n📁 Cluster {cluster_id} ({len(docs)} dokumentů):")
    for doc in docs:
        print(f"   • {doc[:70]}...")

# Vizualizace
clusterer.visualize_clusters(result['embeddings'], result['labels'], documents)

## 5. FAISS - Efektivní vektorové vyhledávání

In [ ]:
import faiss

class FAISSVectorStore:
    """
    Vektorová databáze pomocí FAISS.
    """
    
    def __init__(self, model_name='all-MiniLM-L6-v2'):
        self.model = SentenceTransformer(model_name, device=device)
        self.dimension = self.model.get_sentence_embedding_dimension()
        
        # Inicializace FAISS indexu
        self.index = faiss.IndexFlatIP(self.dimension)  # Inner Product (cosine sim pro normalizované vektory)
        
        self.documents = []
        self.metadata = []
    
    def add(self, documents, metadata=None):
        """
        Přidá dokumenty do indexu.
        """
        # Vytvoření a normalizace embeddings
        embeddings = self.model.encode(documents)
        embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)
        
        # Přidání do FAISS
        self.index.add(embeddings.astype('float32'))
        
        # Uložení dokumentů a metadat
        self.documents.extend(documents)
        if metadata:
            self.metadata.extend(metadata)
        else:
            self.metadata.extend([{} for _ in documents])
        
        print(f"Přidáno {len(documents)} dokumentů. Index obsahuje {self.index.ntotal} vektorů.")
    
    def search(self, query, top_k=5):
        """
        Vyhledá nejpodobnější dokumenty.
        """
        # Embedding dotazu
        query_embedding = self.model.encode([query])
        query_embedding = query_embedding / np.linalg.norm(query_embedding, axis=1, keepdims=True)
        
        # Vyhledání
        scores, indices = self.index.search(query_embedding.astype('float32'), top_k)
        
        results = []
        for score, idx in zip(scores[0], indices[0]):
            if idx < len(self.documents):
                results.append({
                    "document": self.documents[idx],
                    "score": float(score),
                    "metadata": self.metadata[idx],
                    "index": int(idx)
                })
        
        return results
    
    def save(self, path):
        """Uloží index na disk."""
        import pickle
        
        faiss.write_index(self.index, f"{path}.faiss")
        with open(f"{path}.pkl", 'wb') as f:
            pickle.dump({'documents': self.documents, 'metadata': self.metadata}, f)
        print(f"Index uložen do {path}")
    
    def load(self, path):
        """Načte index z disku."""
        import pickle
        
        self.index = faiss.read_index(f"{path}.faiss")
        with open(f"{path}.pkl", 'rb') as f:
            data = pickle.load(f)
            self.documents = data['documents']
            self.metadata = data['metadata']
        print(f"Index načten z {path}. Obsahuje {self.index.ntotal} vektorů.")

# Test
faiss_store = FAISSVectorStore()
print("✅ FAISS VectorStore připraven")

In [ ]:
# Přidání dokumentů
faiss_store.add(documents, metadata)

# Test vyhledávání
query = "Framework pro tvorbu API"
results = faiss_store.search(query, top_k=3)

print(f"\n🔍 Dotaz: {query}")
print("-" * 50)

for i, result in enumerate(results, 1):
    print(f"{i}. [{result['score']:.3f}] {result['document']}")

## 6. Praktická automatizace: FAQ Bot

In [ ]:
class FAQBot:
    """
    Jednoduchý FAQ bot využívající sémantické vyhledávání.
    """
    
    def __init__(self, threshold=0.5):
        self.vector_store = FAISSVectorStore()
        self.threshold = threshold
        self.qa_pairs = []
    
    def add_faq(self, question, answer, tags=None):
        """
        Přidá otázku a odpověď do FAQ.
        """
        self.qa_pairs.append({
            "question": question,
            "answer": answer,
            "tags": tags or []
        })
    
    def build_index(self):
        """
        Vytvoří index z přidaných FAQ.
        """
        questions = [qa['question'] for qa in self.qa_pairs]
        metadata = [
            {"answer": qa['answer'], "tags": qa['tags']}
            for qa in self.qa_pairs
        ]
        
        self.vector_store.add(questions, metadata)
        print(f"Index vytvořen s {len(self.qa_pairs)} FAQ položkami.")
    
    def ask(self, question):
        """
        Odpoví na otázku pomocí FAQ.
        """
        results = self.vector_store.search(question, top_k=3)
        
        if not results or results[0]['score'] < self.threshold:
            return {
                "found": False,
                "message": "Bohužel jsem nenašel odpovídající odpověď. Zkuste formulovat otázku jinak.",
                "suggestions": [r['document'] for r in results]
            }
        
        best_match = results[0]
        
        return {
            "found": True,
            "answer": best_match['metadata']['answer'],
            "matched_question": best_match['document'],
            "confidence": best_match['score'],
            "tags": best_match['metadata'].get('tags', []),
            "alternatives": [
                {"question": r['document'], "score": r['score']}
                for r in results[1:]
            ]
        }

# Vytvoření FAQ bota
faq_bot = FAQBot(threshold=0.4)

# Přidání FAQ
faqs = [
    ("Jak resetovat heslo?", "Pro reset hesla klikněte na 'Zapomenuté heslo' na přihlašovací stránce a zadejte svůj email.", ["účet", "heslo"]),
    ("Jaké jsou způsoby platby?", "Přijímáme platby kartou (Visa, Mastercard), bankovním převodem a Apple Pay.", ["platba", "objednávka"]),
    ("Jak dlouho trvá doručení?", "Standardní doručení trvá 3-5 pracovních dnů. Expresní doručení je možné do 24 hodin.", ["doručení", "objednávka"]),
    ("Jak mohu vrátit zboží?", "Zboží můžete vrátit do 30 dnů od doručení. Vyplňte formulář pro vrácení v sekci 'Moje objednávky'.", ["vrácení", "reklamace"]),
    ("Kde najdu fakturu?", "Fakturu najdete v emailu s potvrzením objednávky nebo v sekci 'Moje objednávky' po přihlášení.", ["faktura", "účet"]),
    ("Jak změnit doručovací adresu?", "Doručovací adresu můžete změnit v nastavení účtu nebo během objednávky před potvrzením.", ["adresa", "doručení"]),
    ("Je možné sledovat zásilku?", "Ano, po odeslání obdržíte email s trackovacím číslem pro sledování zásilky.", ["doručení", "tracking"]),
    ("Jak kontaktovat zákaznickou podporu?", "Podporu můžete kontaktovat emailem na podpora@example.cz nebo telefonicky na 800 123 456.", ["podpora", "kontakt"]),
]

for q, a, tags in faqs:
    faq_bot.add_faq(q, a, tags)

# Vytvoření indexu
faq_bot.build_index()

In [ ]:
# Test FAQ bota
test_questions = [
    "Zapomněl jsem heslo, co mám dělat?",
    "Čím můžu zaplatit?",
    "Kdy mi přijde balík?",
    "Jaké je počasí dnes?"  # Nesouvisející otázka
]

for question in test_questions:
    print(f"\n❓ Otázka: {question}")
    print("=" * 50)
    
    response = faq_bot.ask(question)
    
    if response['found']:
        print(f"✅ Odpověď: {response['answer']}")
        print(f"   Confidence: {response['confidence']:.1%}")
        print(f"   Matched: {response['matched_question']}")
        if response['tags']:
            print(f"   Tags: {', '.join(response['tags'])}")
    else:
        print(f"❌ {response['message']}")
        if response['suggestions']:
            print("   Možná jste měli na mysli:")
            for sugg in response['suggestions']:
                print(f"   • {sugg}")

## 7. Duplicate Detection (Detekce duplicit)

In [ ]:
class DuplicateDetector:
    """
    Detekce duplicitních nebo velmi podobných textů.
    """
    
    def __init__(self, threshold=0.85):
        self.model = SentenceTransformer('all-MiniLM-L6-v2', device=device)
        self.threshold = threshold
    
    def find_duplicates(self, documents):
        """
        Najde skupiny duplicitních dokumentů.
        """
        # Embeddings
        embeddings = self.model.encode(documents, show_progress_bar=True)
        
        # Matice podobnosti
        similarity_matrix = cosine_similarity(embeddings)
        
        # Hledání duplicit
        duplicate_groups = []
        used = set()
        
        for i in range(len(documents)):
            if i in used:
                continue
            
            group = [i]
            for j in range(i + 1, len(documents)):
                if j not in used and similarity_matrix[i, j] >= self.threshold:
                    group.append(j)
                    used.add(j)
            
            if len(group) > 1:
                duplicate_groups.append({
                    "indices": group,
                    "documents": [documents[idx] for idx in group],
                    "similarity": float(np.mean([similarity_matrix[group[0], idx] for idx in group[1:]]))
                })
                used.add(i)
        
        # Unikátní dokumenty
        unique_indices = [i for i in range(len(documents)) if i not in used]
        
        return {
            "duplicate_groups": duplicate_groups,
            "unique_count": len(unique_indices),
            "duplicate_count": len(used),
            "total": len(documents)
        }

# Test
detector = DuplicateDetector(threshold=0.8)

test_docs = [
    "Python je skvělý programovací jazyk.",
    "Python je výborný programovací jazyk pro začátečníky.",  # Podobný
    "JavaScript se používá pro webový vývoj.",
    "Pro vývoj webových stránek se často používá JavaScript.",  # Podobný
    "Machine learning mění svět technologií.",
    "Kočky jsou skvělá domácí zvířata.",
    "Strojové učení revolucionizuje technologický průmysl."  # Podobný k ML
]

result = detector.find_duplicates(test_docs)

print(f"Celkem dokumentů: {result['total']}")
print(f"Unikátních: {result['unique_count']}")
print(f"Duplicitních: {result['duplicate_count']}")

print("\nSkupiny duplicit:")
for i, group in enumerate(result['duplicate_groups'], 1):
    print(f"\n  Skupina {i} (podobnost: {group['similarity']:.1%}):")
    for doc in group['documents']:
        print(f"    • {doc}")

## 8. Text Classification pomocí Embeddings

In [ ]:
class EmbeddingClassifier:
    """
    Zero-shot klasifikace pomocí embeddings.
    """
    
    def __init__(self):
        self.model = SentenceTransformer('all-MiniLM-L6-v2', device=device)
        self.class_embeddings = None
        self.class_names = []
    
    def set_classes(self, class_descriptions: dict):
        """
        Nastaví třídy pomocí textových popisů.
        
        Args:
            class_descriptions: {"název_třídy": "popis třídy"}
        """
        self.class_names = list(class_descriptions.keys())
        descriptions = list(class_descriptions.values())
        
        self.class_embeddings = self.model.encode(descriptions)
        print(f"Nastaveno {len(self.class_names)} tříd: {', '.join(self.class_names)}")
    
    def classify(self, text, return_all=False):
        """
        Klasifikuje text do jedné z tříd.
        """
        if self.class_embeddings is None:
            raise ValueError("Nejprve nastavte třídy pomocí set_classes()")
        
        # Embedding textu
        text_embedding = self.model.encode([text])
        
        # Podobnost s třídami
        similarities = cosine_similarity(text_embedding, self.class_embeddings)[0]
        
        # Nejlepší třída
        best_idx = similarities.argmax()
        
        result = {
            "class": self.class_names[best_idx],
            "confidence": float(similarities[best_idx])
        }
        
        if return_all:
            result["all_scores"] = {
                name: float(score)
                for name, score in zip(self.class_names, similarities)
            }
        
        return result
    
    def classify_batch(self, texts):
        """
        Klasifikuje více textů najednou.
        """
        return [self.classify(text) for text in texts]

# Test
classifier = EmbeddingClassifier()

# Definice tříd
classes = {
    "technology": "Technology, computers, software, programming, AI, internet",
    "sports": "Sports, athletics, games, competitions, teams, players",
    "health": "Health, medicine, fitness, wellness, diseases, treatments",
    "business": "Business, finance, economy, companies, markets, investments",
    "entertainment": "Entertainment, movies, music, celebrities, arts, culture"
}

classifier.set_classes(classes)

In [ ]:
# Test klasifikace
test_texts = [
    "Apple announced new iPhone with improved camera and longer battery life.",
    "The team won the championship after an exciting final match.",
    "New study shows benefits of regular exercise for heart health.",
    "Stock market reached record highs amid positive economic data.",
    "The new movie broke box office records in its opening weekend."
]

print("Výsledky klasifikace:")
print("=" * 60)

for text in test_texts:
    result = classifier.classify(text, return_all=True)
    
    print(f"\n📝 {text[:60]}...")
    print(f"   ➡️ {result['class'].upper()} ({result['confidence']:.1%})")
    
    # Top 3 skóre
    sorted_scores = sorted(result['all_scores'].items(), key=lambda x: -x[1])[:3]
    scores_str = ", ".join([f"{k}: {v:.1%}" for k, v in sorted_scores])
    print(f"   Scores: {scores_str}")

## Shrnutí

V tomto notebooku jsme se naučili:

1. **Text Embeddings** - vektorové reprezentace textu pomocí Sentence Transformers
2. **Sémantické vyhledávání** - hledání dokumentů na základě významu
3. **Vícejazyčné embeddings** - práce s více jazyky
4. **Clustering dokumentů** - automatické seskupování podobných textů
5. **FAISS** - efektivní vektorové vyhledávání pro velké kolekce
6. **FAQ Bot** - praktická aplikace sémantického vyhledávání
7. **Detekce duplicit** - hledání podobných dokumentů
8. **Zero-shot klasifikace** - klasifikace bez trénování

### Další kroky
- Notebook 12: Fine-tuning modelů